In [1]:
!pip install pynput
!pip install  torch_lr_finder
!pip install pytorch_lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.2/182.2 kB 8.9 MB/s eta 0:00:00
  Created wheel for evdev: filename=evdev-1.6.1-cp310-cp310-linux_x86_64.whl size=45847 sha256=ac1e082a31038dabff1800ec9a8ae9d7a8d4214d99241b3bb0c6ee76a257f20d
  Stored in directory: /root/.cache/pip/wheels/bb/6e/51/0c0f5677bd58c532c42e13e7d93803fbf8d5d3bf5bfc5b0220
Successfully built evdev


In [2]:
# !mkdir store

In [2]:
!ls store

'epoch=29-step=15027.ckpt'     'epoch=36-step=19166.ckpt'
'epoch=31-step=15037.ckpt'     'epoch=37-step=19684.ckpt'
'epoch=32-step=15042.ckpt'     'epoch=38-step=20202.ckpt'
'epoch=35-step=18648.ckpt'     'epoch=4-step=25.ckpt'
'epoch=36-step=19166-v1.ckpt'


In [4]:
!rm -r /kaggle/working/yolov3_object_detection/
%cd /kaggle/working
!git clone https://github.com/venkyyuvy/yolov3_object_detection.git
%cd /kaggle/working/yolov3_object_detection
# !git checkout modified_callback
!python main.py

/kaggle/working
Cloning into 'yolov3_object_detection'...
remote: Enumerating objects: 185, done.
remote: Counting objects: 100% (185/185), done.
remote: Compressing objects: 100% (123/123), done.
remote: Total 185 (delta 114), reused 127 (delta 62), pack-reused 0
Receiving objects: 100% (185/185), 278.18 KiB | 3.39 MiB/s, done.
Resolving deltas: 100% (114/114), done.
/kaggle/working/yolov3_object_detection
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/sit

In [29]:
from model import YOLOv3
import torch
fname = '../store/epoch=38-step=20202.ckpt'
checkpoint = torch.load(fname, map_location=torch.device('cuda'))
model_state_dict = checkpoint['state_dict']
model = YOLOv3(num_classes=20)
model.load_state_dict(model_state_dict)

<All keys matched successfully>

In [30]:
import config
import utils
from dataset import YOLODataModule

yolo_datamodule = YOLODataModule(
        train_csv_path=config.DATASET + "/train.csv",
        test_csv_path=config.DATASET + "/test.csv",
    )

yolo_datamodule.setup()

In [34]:
model.eval()
model = model.to(config.DEVICE)
pred_boxes, true_boxes = utils.get_evaluation_bboxes(
        loader=yolo_datamodule.test_dataloader(),
        model=model,
        iou_threshold=config.NMS_IOU_THRESH,
        anchors=config.ANCHORS,
        threshold=config.CONF_THRESHOLD,
    )

100%|██████████| 310/310 [22:14<00:00,  4.31s/it]


In [36]:
from utils import mean_average_precision
map_val = mean_average_precision(
    pred_boxes=pred_boxes,
    true_boxes=true_boxes,
    iou_threshold=config.MAP_IOU_THRESH,
    box_format="midpoint",
    num_classes=config.NUM_CLASSES,
).item()
print("+++ MAP: ", map_val)


100%|██████████| 20/20 [03:44<00:00, 11.24s/it]

+++ MAP:  0.570273756980896
